In [37]:
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import pandas as pd
from enum import Enum
import graphviz as gv
from langchain.prompts.chat import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
import json
from IPython.display import JSON
from tqdm.notebook import tqdm

load_dotenv()

True

In [38]:
llm = ChatOpenAI(model_name="gpt-4")

# A List of Haunted Houses
Let's try a more "top-down" approach, where we first generate a dataset of haunted houses that we will later use to populate with ghosts
individually.

In [62]:
class HauntedHouse(BaseModel):
    name: str
    address: str
    type: str
    number_of_ghosts: int
    year_built: int


class HauntedHouseListings(BaseModel):
    location: str
    haunted_houses: list[HauntedHouse]


pydantic_parser = PydanticOutputParser(pydantic_object=HauntedHouseListings)

PROMPT_FORMAT = """You are a creative AI who loves the macabre.
Create {house_count} haunted houses in {location} and express these as a list.
The number of ghosts that reside in each house should follow a Poisson distribution with mean {mean_ghost_count}.
{format_instructions}
The output should not deviate from this format in any way and every house should be listed.
No comments or ellipsis should appear in the output.
"""

prompt = PromptTemplate(
    template=PROMPT_FORMAT,
    input_variables=["location", "house_count", "mean_ghost_count"],
    partial_variables={
        "format_instructions": pydantic_parser.get_format_instructions()
    },
)

chain = LLMChain(llm=llm, prompt=prompt, output_parser=pydantic_parser, verbose=True)

haunted_houses = chain.run(
    location="Covington Georgia", house_count=50, mean_ghost_count=2.5
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a creative AI who loves the macabre.
Create 50 haunted houses in Covington Georgia and express these as a list.
The number of ghosts that reside in each house should follow a Poisson distribution with mean 2.5.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"HauntedHouse": {"properties": {"name": {"title": "Name", "type": "string"}, "address": {"title": "Address", "type": "string"}, "type": {"title": "Type", "type": "string"}, "number_of_ghosts": {"title": "Number Of Ghosts", "type": "integer"}, "year_built": {"

In [105]:
with open("haunted_houses.json", "w") as f:
    f.write(haunted_houses.model_dump_json())

In [70]:
haunted_houses_df = pd.DataFrame(
    map(BaseModel.model_dump, haunted_houses.haunted_houses)
)
haunted_houses_df

,name,address,type,number_of_ghosts,year_built
0,The Crimson Manor,1 Creepy Lane,Manor,2,1888
1,The Spectral Cottage,2 Ghostly Grove,Cottage,3,1805
2,The Shadowy Mansion,3 Phantom Path,Mansion,1,1820
3,The Haunted Hovel,4 Spooky Street,Hovel,4,1900
4,The Eerie Estate,5 Frightful Fairway,Estate,3,1850
5,The Ghostly Gables,6 Terror Trail,Gables,2,1875
6,The Spooky Shack,7 Haunted Highway,Shack,5,1915
7,The Chilling Chateau,8 Ghoul Grove,Chateau,3,1895
8,The Frightful Farmhouse,9 Phantom Parkway,Farmhouse,2,1860
9,The Terrifying Townhouse,10 Eerie Estate,Townhouse,1,1925


In [71]:
haunted_houses_df.number_of_ghosts.mean()

2.64

In [72]:
haunted_houses_df.number_of_ghosts.sum()

132

## Generating Ghosts

### Reintroducing our Scariness Scale and Moving On Conditions

In [75]:
with open("scariness_definition.json") as f:
    scariness_scale = json.load(f)
scariness_scale

[{'level': 1,
  'name': 'Benevolent Specter',
  'description': 'Exhibits non-threatening, socially positive behavior patterns.'},
 {'level': 2,
  'name': 'Auditory Manifestation',
  'description': 'Engages primarily in acoustic disturbances, evoking mild unease.'},
 {'level': 3,
  'name': 'Object Displacement Phenomenon',
  'description': 'Demonstrates ability for limited physical interaction, typically harmless.'},
 {'level': 4,
  'name': 'Thermokinetic Fluctuation',
  'description': 'Presence is associated with noticeable temperature drops and psychosomatic discomfort.'},
 {'level': 5,
  'name': 'Visual Entity',
  'description': 'Capable of producing visual hallucinations without direct physical interaction.'},
 {'level': 6,
  'name': 'Kinetic Manifestation',
  'description': 'Shows ability to interact significantly with its environment, causing observable disruptions.'},
 {'level': 7,
  'name': 'Aggressive Entity',
  'description': 'Exhibits hostile behavior patterns, induces psycho

In [78]:
with open("moving_on.json") as f:
    moving_on_conditions = json.load(f)
moving_on_conditions

[{'name': 'Resolution of Unfinished Business',
  'description': 'Ghosts may need to resolve some issue or unfinished business from their life before they can move on.',
  'code': 'ROUB'},
 {'name': 'Acceptance of Death',
  'description': 'Some ghosts may not even realize they are dead and may need to come to terms with their own death.',
  'code': 'AOD'},
 {'name': 'Revenge or Justice',
  'description': 'In some stories, a ghost may need to see justice served or revenge taken before they can rest.',
  'code': 'ROJ'},
 {'name': 'Proper Burial',
  'description': 'In many cultures, ghosts cannot move on until they receive a proper burial or funeral rites.',
  'code': 'PB'},
 {'name': 'Reconciliation',
  'description': 'The ghost may need to reconcile with someone they wronged in life or someone who wronged them.',
  'code': 'REC'},
 {'name': 'Retrieval of Lost Items',
  'description': 'Sometimes, ghosts are tied to this realm due to lost or stolen items that they valued greatly.',
  'code

### The ghost model and generating prompt

In [83]:
class Ghost(BaseModel):
    name_in_afterlife: str
    description_in_afterlife: str
    name_when_living: str
    description_when_living: str
    type_of_ghost: str
    scariness: int
    sex: str
    year_of_birth: int
    year_of_demise: int
    description_of_demise: str
    moving_on_condition_code: str
    preferred_victims_to_scare: str
    preferred_method_to_scare: str


class GhostList(BaseModel):
    ghosts: list[Ghost]


pydantic_parser = PydanticOutputParser(pydantic_object=GhostList)

PROMPT_FORMAT = """You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of ghosts who all reside within a haunted house with the following particulars.
```{house_info}```
Most ghosts will not be very scary, but some will be extremely scary.
The scariness of each ghost will conform to the following scale.
```{scariness_scale}```
The moving on condition for each ghost may be gotten from the following.
```{moving_on_conditions}```
{format_instructions}
The output should not deviate from this format in any way and every ghost should be listed.
No comments or ellipsis should appear in the output.
"""

prompt = PromptTemplate(
    template=PROMPT_FORMAT,
    input_variables=["house_info"],
    partial_variables={
        "scariness_scale": str(scariness_scale),
        "moving_on_conditions": str(moving_on_conditions),
        "format_instructions": pydantic_parser.get_format_instructions(),
    },
)
chain = LLMChain(llm=llm, prompt=prompt, output_parser=pydantic_parser, verbose=True)

### Try the First Haunted House as an Example

In [84]:
ghost_list = chain.run(house_info=haunted_houses.haunted_houses[0].model_dump_json())



> Entering new LLMChain chain...
Prompt after formatting:
You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of ghosts who all reside within a haunted house with the following particulars.
```{"name":"The Crimson Manor","address":"1 Creepy Lane","type":"Manor","number_of_ghosts":2,"year_built":1888}```
Most ghosts will not be very scary, but some will be extremely scary.
The scariness of each ghost will conform to the following scale.
```[{'level': 1, 'name': 'Benevolent Specter', 'description': 'Exhibits non-threatening, socially positive behavior patterns.'}, {'level': 2, 'name': 'Auditory Manifestation', 'description': 'Engages primarily in acoustic disturbances, evoking mild unease.'}, {'level': 3, 'name': 'Object Displacement Phenomenon', 'description': 'Demonstrates ability for limited physical interaction, typically harmless.'}, {'level': 4, 'name': 'Thermokinetic Fluctuation', 'description': 'Presence is associated with noticeable te

In [85]:
ghosts_df = pd.DataFrame(map(BaseModel.model_dump, ghost_list.ghosts))
display(JSON(json.loads(haunted_houses.haunted_houses[0].model_dump_json())), ghosts_df)

<IPython.core.display.JSON object>

,name_in_afterlife,description_in_afterlife,name_when_living,description_when_living,type_of_ghost,scariness,sex,year_of_birth,year_of_demise,description_of_demise,moving_on_condition_code,preferred_victims_to_scare,preferred_method_to_scare
0,Crimson Lady,A female specter often spotted drifting throug...,Lady Elizabeth Wraithwood,The lady of the manor known for her fiery temp...,Visual Entity,5,Female,1848,1888,"Died under mysterious circumstances, her body ...",UT,Visitors who dare to enter her private chambers.,Materializing in her crimson gown before disap...
1,Wailing Child,A child ghost whose melancholic wails echo thr...,Timothy Wraithwood,"The youngest son of the Wraithwood family, a p...",Auditory Manifestation,2,Male,1882,1888,"Died of a sudden illness, his wails for his mo...",LOV,People who ignore or seem indifferent to his w...,"His wails growing louder and more desperate, e..."


### Let's Run Them All Now.

In [90]:
from typing import Optional


# Let define a function to automatically retry if there's any exception.
def _generate_with_retry(
    haunted_house: HauntedHouse, max_retries: int = 5
) -> tuple[HauntedHouse, Optional[GhostList]]:
    for retry in range(max_retries):
        try:
            ghost_list = chain.run(house_info=haunted_house.model_dump_json())
            return haunted_house, ghost_list
        except Exception as e:
            print(f"retrying '{haunted_house}' due to '{e}'")
    return haunted_house, None


houses_ghost_lists = list(
    map(_generate_with_retry, tqdm(haunted_houses.haunted_houses))
)

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of ghosts who all reside within a haunted house with the following particulars.
```{"name":"The Crimson Manor","address":"1 Creepy Lane","type":"Manor","number_of_ghosts":2,"year_built":1888}```
Most ghosts will not be very scary, but some will be extremely scary.
The scariness of each ghost will conform to the following scale.
```[{'level': 1, 'name': 'Benevolent Specter', 'description': 'Exhibits non-threatening, socially positive behavior patterns.'}, {'level': 2, 'name': 'Auditory Manifestation', 'description': 'Engages primarily in acoustic disturbances, evoking mild unease.'}, {'level': 3, 'name': 'Object Displacement Phenomenon', 'description': 'Demonstrates ability for limited physical interaction, typically harmless.'}, {'level': 4, 'name': 'Thermokinetic Fluctuation', 'description': 'Presence is associated with noticeable te

## Merging the Haunted Houses and Ghosts Models

In [95]:
class PopulatedHauntedHouse(BaseModel):
    name: str
    address: str
    type: str
    ghosts: list[Ghost]
    year_built: int


class PopulatedHauntedHouseListings(BaseModel):
    location: str
    haunted_houses: list[PopulatedHauntedHouse]

In [96]:
populated_haunted_house_listings = PopulatedHauntedHouseListings(
    location=haunted_houses.location,
    haunted_houses=[
        PopulatedHauntedHouse(
            **haunted_house.model_dump() | dict(ghosts=ghost_list.ghosts)
        )
        for haunted_house, ghost_list in houses_ghost_lists
    ],
)

In [104]:
with open("populated_haunted_house_listings.json", "w", encoding="utf-8") as f:
    json.dump(
        populated_haunted_house_listings.model_dump_json(),
        f,
        ensure_ascii=True,
        indent=4,
    )

In [98]:
# with open("populated_haunted_house_listings.json", "r") as f:
#     loaded_house_listings = json.load(f)
# loaded_house_listings = json.loads(loaded_house_listings)
# loaded_house_listings

### The Ghosts DataFrame

In [99]:
ghosts_df = pd.DataFrame(
    h.model_dump(exclude="ghosts") | g.model_dump()
    for h in populated_haunted_house_listings.haunted_houses
    for g in h.ghosts
)
ghosts_df

,name,address,type,year_built,name_in_afterlife,description_in_afterlife,name_when_living,description_when_living,type_of_ghost,scariness,sex,year_of_birth,year_of_demise,description_of_demise,moving_on_condition_code,preferred_victims_to_scare,preferred_method_to_scare
0,The Crimson Manor,1 Creepy Lane,Manor,1888,Lady in White,A spectral woman dressed in a flowing white go...,Eleanor Winthrop,"A beautiful, kind-hearted noblewoman, who trag...",Visual Entity,5,Female,1850,1878,"Died of a broken heart, after waiting for year...",ROUB,Young men who remind her of her lost love.,Appearing in mirrors and crying out in the night.
1,The Crimson Manor,1 Creepy Lane,Manor,1888,The Phantom Butler,"A ghostly figure in servant's attire, often se...",Alfred Pennington,A loyal butler in service of the Winthrop fami...,Object Displacement Phenomenon,3,Male,1820,1899,"Died in his sleep in the butler's quarters, ha...",ROLI,Anyone who disrupts the orderliness of the manor.,Moving objects around to instill a sense of di...
2,The Spectral Cottage,2 Ghostly Grove,Cottage,1805,Whispering Winston,A spectral figure seen only in the corner of o...,Winston Hawthorne,A quiet scholar with a love for literature and...,Auditory Manifestation,2,Male,1760,1830,"Died peacefully in his sleep, in the very same...",ROUB,Lone researchers in the library,"Soft whispers, rustling pages, and echoing foo..."
3,The Spectral Cottage,2 Ghostly Grove,Cottage,1805,Lady Lament,"A sorrowful entity, often seen as a white figu...",Lady Constance,An aristocratic woman known for her beauty and...,Visual Entity,5,Female,1785,1810,Died of a broken heart after her lover was los...,LOV,Young couples,"Sudden appearances, mournful wails, and cold s..."
4,The Spectral Cottage,2 Ghostly Grove,Cottage,1805,Tormented Thomas,An aggressive apparition with a penchant for c...,Thomas Blackwood,A hardworking blacksmith known for his fiery t...,Kinetic Manifestation,6,Male,1775,1825,"Died in a tragic blacksmithing accident, his s...",ROJ,Anyone who dares to touch his tools,"Violently throwing objects, loud banging sound..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,The Beastly Bakery,48 Spectre Street,Bakery,1890,Beryl the Breadloaf Banshee,"A shrieking specter, often seen carrying a pha...",Beryl Baker,"Beryl was a stern, no-nonsense woman who ran t...",Auditory Manifestation,2,Female,1855,1898,Died of heartbreak after her precious bakery w...,AOD,"Anyone who dares to waste food, especially bread.",Screaming at the top of her ghostly lungs.
132,The Beastly Bakery,48 Spectre Street,Bakery,1890,Doughboy Danny,A playful specter often seen tossing phantom d...,Daniel Dougherty,"Daniel was a cheeky, young apprentice baker, m...",Kinetic Manifestation,6,Male,1870,1890,Died from a sudden illness before he could ful...,ROUB,Anyone who doesn't appreciate the art of baking.,Tossing phantom dough at unsuspecting victims.
133,The Spooky Spa,49 Phantom Parkway,Spa,1875,Spa Specter,"Lingers around the steam rooms, often causing ...",Marianne Leclerc,"Was the original owner of the spa, a gentle wo...",Thermokinetic Fluctuation,4,Female,1825,1879,Died tragically in a fire that broke out in th...,ROUB,Spa guests who do not respect the tranquility ...,Lowers the temperature in the steam rooms to a...
134,The Frightening Factory,50 Ghostly Grove,Factory,1905,Wailing Wendy,A sorrowful specter that is often heard weepin...,Wendy Worksworth,A hardworking factory worker who was known for...,Auditory Manifestation,2,Female,1880,1905,"Tragically died in a factory accident, crushed...",ROUB,Factory managers and supervisors,Cries and wails that echo throughout the factory


In [103]:
ghosts_df[ghosts_df.scariness == 10]

,name,address,type,year_built,name_in_afterlife,description_in_afterlife,name_when_living,description_when_living,type_of_ghost,scariness,sex,year_of_birth,year_of_demise,description_of_demise,moving_on_condition_code,preferred_victims_to_scare,preferred_method_to_scare
11,The Haunted Hovel,4 Spooky Street,Hovel,1900,Terrifying Tilda,A ghost that preys on the deepest fears of the...,Matilda Evans,A stern schoolteacher known for her terrifying...,Poltergeist Phenomenon,10,Female,1860,1901,"Died under mysterious circumstances, her ghost...",UT,Those who show disrespect or disobedience.,Manifesting their deepest fears into reality.
21,The Spooky Shack,7 Haunted Highway,Shack,1915,Raging Robert,A ghost who displays extreme physical interact...,Robert K. Davis,A blacksmith known for his violent temper.,Poltergeist Phenomenon,10,Male,1845,1915,"Killed in a bar fight, his anger leading to hi...",ROJ,Those who are quick to anger.,Violently moving objects and creating havoc.
29,The Terrifying Townhouse,10 Eerie Estate,Townhouse,1925,Ghastly George,"A terrifying figure with a dark, shadowy form.",George Grimshaw,"A cruel and bitter man, infamous for his temper.",Poltergeist Phenomenon,10,Male,1875,1925,Died in a fit of rage during a heated argument.,ROJ,Anyone who dares to cross him,Throws objects and creates chaos in a fit of s...
43,The Appalling Abode,14 Frightful Field,Abode,1920,Poltergeist Paul,His violent outbursts can cause significant da...,Paul Mayhem,An aggressive man known for his violent tenden...,Poltergeist Phenomenon,10,Male,1850,1920,Killed in a brawl.,EXO,Bullies and aggressive individuals.,Physical disruptions and violent outbursts.
51,The Horrid Homestead,18 Wraith Way,Homestead,1870,Spiteful Samuel,A vengeful spirit causing disturbances and hav...,Samuel Whitmore,The stern and quick-tempered master of the hou...,Poltergeist Phenomenon,10,Male,1815,1870,Found dead in his library with a knife in his ...,ROJ,Anyone who dares to touch his collection of bo...,"Throwing books off shelves, toppling furniture..."
67,The Fearsome Factory,24 Phantom Place,Factory,1940,Frightful Fred,An aggressive apparition who throws objects ar...,Fred Foster,A grumpy factory worker known for his temper.,Poltergeist Phenomenon,10,Male,1890,1943,Died in a fight with a co-worker.,ROJ,Factory workers,Throws objects and creates loud noises.
91,The Macabre Museum,33 Spectre Street,Museum,1900,The Specter of Spectre Street,Known to roam the corridors of The Macabre Mus...,Arthur Pendleton,A curious historian and the original curator o...,Poltergeist Phenomenon,10,male,1850,1910,Died under mysterious circumstances in the mus...,UT,Anyone who dares to stay in the museum after d...,"Moving museum exhibits, creating sudden temper..."
111,The Fearsome Fort,41 Phantom Parkway,Fort,1860,Colonel Phantom,"A tall, imposing figure in a military uniform....",Colonel Jonathan Haverhill,A stern and strict military man who commanded ...,Poltergeist Phenomenon,10,Male,1820,1862,"Killed in battle, his body was never properly ...",PB,People who disrespect the fort,"Manifests as a terrifying apparition, causing ..."
117,The Lurid Library,42 Ghostly Grove,Library,1920,The Poltergeist of Prose,A terrifying entity that violently interacts w...,Emily Wordsworth,An author who never achieved the recognition s...,Poltergeist Phenomenon,10,Female,1860,1920,Died of heartbreak after her book was rejected...,ROJ,Critics and successful authors,Throwing her unpublished manuscripts and causi...


In [100]:
ghosts_df.groupby("scariness").name_in_afterlife.count()

scariness
1      8
2     39
3     11
4     19
5     27
6     16
7      6
9      1
10     9
Name: name_in_afterlife, dtype: int64

In [102]:
ghosts_df.groupby("sex").scariness.mean()

sex
Female     4.09375
Male       4.15493
male      10.00000
Name: scariness, dtype: float64

In [ ]:
# import asyncio


# async def async_generate(achain: LLMChain, haunted_house: HauntedHouse):
#     response = await achain.arun(house_info=haunted_house.model_dump_json())
#     return haunted_house, response


# async def generate_concurrently():
#     _llm = ChatOpenAI(model_name="gpt-4")
#     _chain = LLMChain(llm=_llm, prompt=prompt, output_parser=pydantic_parser)
#     tasks = [
#         await async_generate(achain=_chain, haunted_house=haunted_house)
#         for haunted_house in tqdm(haunted_houses.haunted_houses[:2])
#     ]
#     return await asyncio.gather(*tasks)


# houses_and_ghosts_lists = await generate_concurrently()